<a href="https://colab.research.google.com/github/BrunoGeraldine/B3_Prediction_2023/blob/main/notebook/B3_LTSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# B3_Prediction_2023 → LTSM

## Bibliotecas e help functions

In [ ]:
# Import
import sidetable
import numpy                 as np
import pandas                as pd
import seaborn               as sns
import matplotlib.pyplot     as plt

from tensorflow.keras.models import Sequential
from sklearn.preprocessing   import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, LSTM, Dropout


In [ ]:
#!pip install sidetable

In [ ]:
## Help Functions

# Supress scientific Notation
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Read data

In [ ]:
df = pd.read_parquet('/content/all_bovespa.parquet')

In [ ]:
df.head()

# Action_Filter

In [ ]:
df_acao = df.copy()

In [ ]:
sigla = 'PETR4'
df_acao = df[df['sigla_acao'] == sigla ]

In [ ]:
df_acao

In [ ]:
# Selecionando os campos que vamos análisar
df_acao_fech = df_acao[['data_pregao', 'preco_fech']]


In [ ]:
# Ajustando o indice pela data
df_acao_fech = df_acao_fech.set_index(pd.DatetimeIndex(df_acao_fech['data_pregao'].values))

In [ ]:
# Eliminano a coluna 'data_pregao'

df_acao_fech.drop(columns=['data_pregao'], inplace=True)

# EDA → Action

## Verificar quantidade de linhas

In [ ]:
print( f'Number of Rows: {df_acao_fech.shape[0]}' )
print( f'Number of Columns: {df_acao_fech.shape[1]}' ) 

## Data types

In [ ]:
df_acao_fech.dtypes

## Check NA

In [ ]:
df_acao_fech.stb.missing()

## Data description

In [ ]:
df_acao_fech.describe()

## Grafico dos valores de fechamento

In [ ]:
# Plotando o comportamento da ação ao longo do periodo 2019 > 2023

plt.figure(figsize=(25,8))
plt.title('Preco de fechamento 2019 à 2023')
plt.plot(df_acao_fech['preco_fech'], color='darkslateblue' )
plt.xlabel('data');


# Train_Test_Split

In [ ]:
# coletando a quantidade de linhas
qtd_linhas = len(df_acao_fech)

qtd_linhas_treino =round(.70 * qtd_linhas)

qtd_linhas_test = qtd_linhas - qtd_linhas_treino

info = (
    f'Linhas de treino = 0:{qtd_linhas_treino}, '
    f'linhas de teste = {qtd_linhas_treino}:{qtd_linhas_treino+qtd_linhas_test}'
)

info

# Data Modeling

In [ ]:
# Normalização dos dados

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_acao_fech)

In [ ]:
df_scaled

## Select Train_data and Test_data